In [ ]:
from datetime import datetime as dt
from datetime import timedelta
import logging
import importlib
import urllib
import unicodecsv as csv
from lxml import html
import requests
import pandas as pd
import matplotlib.pyplot as plt, numpy as np
import json
import sys
from requests.auth import HTTPProxyAuth
import time
import os
%matplotlib inline

In [ ]:
#Script is written right now for before cleaned, single region. Must adapt for cleaned and multi-region.
#This will include changing "price" to "rent"
#QUESTION: Unclear what is actually being counted in 'score' variable

#cleandf = pd.read_csv('C:\\Users\\james\\Documents\\Berkeley_Docs\\Spring_17_Courses\\CP290 Data Lab\\github\\scraper2\\shared_room_scraper\\sfbay_merged_2017_04_10.csv', header=0) #just SF dataframe
df = pd.read_csv('C:\\Users\\james\\Documents\\Berkeley_Docs\\Spring_17_Courses\\CP290 Data Lab\\github\\scraper2\\shared_room_scraper\\all_data_merged_2017_04_six_nine.csv',error_bad_lines=False, header=0) #all listings dataframe
#determine rent per square foot


In [ ]:
df.shape

In [ ]:
df.loc[df.lat == 99, 'lat'] = 0
df.loc[df.lng == 99, 'lng'] = 0
df.loc[df.sqft == 0, 'sqft'] = np.nan

In [ ]:
df.tail()

In [ ]:
#For any duplicate post, we want to keep the version with the most information. Therefore, assign a score to each post and 
#give one point for square footage, lat, long and price
#OLD FORMULA
#cleandf['score']=cleandf.astype(bool).sum(axis=1)

df['price_exists'] = df['price']>0
df['sqft_exists'] = df['sqft']>0
df['lat_exists'] = df['lat']>0
df['lng_exists'] = df['lng']<0
df['score'] = df[['price_exists','sqft_exists','lat_exists','lng_exists']].astype(bool).sum(axis=1)

In [ ]:
#Sort rows by score
df = df.sort_values(by='score',ascending=False)

In [ ]:
df.shape

In [ ]:
df.tail()

In [ ]:
#List as 'true' if the PID, body_text, or the title are duplicates
df['body_text_duplicate']=df.duplicated('body_text')
df['title_duplicate']=df.duplicated('title')
df['PID_duplicate']=df.duplicated('pid')


In [ ]:
df_dedupe1 = df.drop_duplicates('pid')
df_dedupe1.shape

In [ ]:
#Keep rows with highest score. Drop rows with duplicate PIDs, Duplicate Titles, Duplicate Posting Bodies
df_dedupe2 = df_dedupe1.drop_duplicates('body_text')
df_dedupe2.shape

In [ ]:
df_unique = df_dedupe2.drop_duplicates('title')
df_unique.shape

### Quick bar chart to see impact of deduplication on sample size

In [ ]:
y = [len(df),len(df_dedupe1), len(df_dedupe2), len(df_unique)]

In [ ]:
labels = ['original', 'dedupe-pid', 'dedupe-body', 'dedupe-title']
x = [1,2,3,4]

In [ ]:
plt.figure(figsize=(10,6) )
plt.suptitle('Deduplication Sample Size')
plt.xlabel('Deduplication Phase')
plt.ylabel('Number of Listings')
plt.xticks(x, labels)
ax = plt.bar(x, y, alpha=.4, color='cyan', align='center')
plt.show()

## Creating new fields for analysis

In [ ]:
df_unique['dt'] = pd.to_datetime(df_unique['dt'], format='%Y-%m-%d')
df_unique['region'] = df_unique['url'].str.extract('http://(.*).craigslist.org', expand=False)
df_unique['day_of_week'] = df_unique['dt'].apply(lambda x: x.weekday())
df_unique['rent_sqft'] = df_unique['price'] / df_unique['sqft']

In [ ]:
df_unique['date'] = pd.DatetimeIndex(df_unique.dt).normalize()

In [ ]:
#counting listings per day (right now inlcudes time, need to remove)
listings_per_date = df_unique['date'].value_counts()

In [ ]:
listings_per_date

## Descriptive Stats and More Filtering

In [ ]:
df_unique.describe()

In [ ]:
unique_wprice = df_unique[df_unique['price'] > 0]

In [ ]:
# in this cell, define the values by which we will filter the 3 columns. This will vary depending on the sample we're looking at. 
upper_percentile = 0.9965
lower_percentile = 0.03

# how many rows would be within the upper and lower percentiles?
upper = int(len(unique_wprice) * upper_percentile)
lower = int(len(unique_wprice) * lower_percentile)

# get the rent/sqft values at the upper and lower percentiles
rent_sqft_sorted = unique_wprice['rent_sqft'].sort_values(ascending=True, inplace=False)
upper_rent_sqft = rent_sqft_sorted.iloc[upper]
lower_rent_sqft = rent_sqft_sorted.iloc[lower]

# get the rent values at the upper and lower percentiles
rent_sorted = unique_wprice['price'].sort_values(ascending=True, inplace=False)
upper_rent = rent_sorted.iloc[upper]
lower_rent = rent_sorted.iloc[lower]

# get the sqft values at the upper and lower percentiles
sqft_sorted = unique_wprice['sqft'].sort_values(ascending=True, inplace=False)
upper_sqft = sqft_sorted.iloc[upper]
lower_sqft = sqft_sorted.iloc[lower]

print('valid rent_sqft range:', [lower_rent_sqft, upper_rent_sqft])
print('valid rent range:', [lower_rent, upper_rent])
print('valid sqft range:', [lower_sqft, upper_sqft])

In [ ]:
#Filtering out rows with unreasonable rent prices 
rent_mask = (unique_wprice['price'] > lower_rent) & (unique_wprice['price'] < upper_rent)

In [ ]:
filtered_listings = pd.DataFrame(unique_wprice[rent_mask])

## Filtering out listings with "0" latitudes

In [ ]:
lat_mask = (filtered_listings['lat'] > 0)

In [ ]:
filtered_listings = pd.DataFrame(filtered_listings[lat_mask])

In [ ]:
filtered_listings.describe()

## Getting FIPS (Need to add to Istanbul script, too slow here)

In [ ]:
# Creating columns for FIPS
filtered_listings['FIPS'] = None
filtered_listings['state'] = None
filtered_listings['county'] = None

In [ ]:
#each row to the API call

for i, row in filtered_listings.iterrows():
    url = 'http://data.fcc.gov/api/block/find?format=json&latitude={0}&longitude={1}'
    request = url.format(row['lat'], row['lng'])
    response = requests.get(request)
    data = response.json()
    filtered_listings.loc[i,['FIPS','state','county']] = [data['Block']['FIPS'], data['State']['code'], data['County']['name']]

In [ ]:
[data['Block']['FIPS'], data['State']['code'], data['County']['name']]

### Filtering by amenities

In [ ]:
df_rooms = filtered_listings[filtered_listings['room_known']==True]

In [ ]:
df_rooms[df_rooms['private_room']==True].describe()

In [ ]:
df_rooms[df_rooms['private_room']==False].describe()

## Viewing Results by Region

In [ ]:
# Here we're pulling select regions to do neighborhood analysis, as well as intraregional comparisons to the conventional 
# listings and inter-regional comparisons. 

#Top 30 regions by unique postings
filtered_listings['region'].value_counts().head(30)

In [ ]:
nylistings = filtered_listings[filtered_listings['region']=='newyork']

In [ ]:
grouped = nylistings.groupby(by='neighb')

In [ ]:
counted = grouped.count()

In [ ]:
counted.reset_index(inplace=True)

In [ ]:
goodNeighbs = counted['neighb'][counted['pid'] > 20].values

In [ ]:
top_nyneighbs = nylistings[nylistings['neighb'].isin(goodNeighbs)]

In [ ]:
top_nyneighbs.groupby('neighb')['price'].median()

In [ ]:
sflistings = filtered_listings[filtered_listings['region']=='sfbay']
sfgrouped = sflistings.groupby(by='neighb')
sfcounted = sfgrouped.count()
sfcounted.reset_index(inplace=True)
sfgoodNeighbs = sfcounted['neighb'][sfcounted['pid'] > 20].values
top_sfneighbs = sflistings[sflistings['neighb'].isin(sfgoodNeighbs)]

In [ ]:
top_sfneighbs.groupby('neighb')['price'].median()

## Visualization

In [ ]:
x = filtered_listings['price']

In [ ]:
x = x.astype('int')

In [ ]:
plt.figure(1, figsize=(10,6) )
plt.suptitle('Rent Price Distribution')
plt.xlabel('Price')
plt.ylabel('Number of Listings')
ax = plt.hist(x, bins=50, alpha=.4, color='r')
plt.show()

### Regional Comparisons

In [ ]:
# Here we're pulling select regions to do neighborhood analysis, as well as intraregional comparisons to the conventional 
# listings and inter-regional comparisons. 

In [ ]:
filtered_listings

## Counting Words in body text

In [ ]:
np.sum(filtered_listings['body_text'].str.contains('professional'))

In [ ]:
np.sum(filtered_listings['body_text'].str.contains('young'))

In [ ]:
np.sum(filtered_listings['body_text'].str.contains('20s'))

In [ ]:
np.sum(filtered_listings['body_text'].str.contains('30s'))

In [ ]:
#filtered_listings.to_csv('sfbay_filtered4_10.csv')

In [ ]:
# calculate the ratios of unique to duplicate listings for each region
listings_ratios = pd.DataFrame()

# number of total listings for each region
listings_ratios['cleandf'] = cleandf['region'].value_counts()

# number of duplicate listings for each region (ie, listings that share a pid with at least one other listing)
listings_ratios['duplicate_listings'] = duplicate_listings['region'].value_counts()

# number of unique listings for the region (ie, none share a pid with another listing)
listings_ratios['unique_listings'] = unique_listings['region'].value_counts()

# percent of this region's listings that are duplicates
listings_ratios['duplicate_ratio'] = listings_ratios['duplicate_listings'] / listings_ratios['all_listings']

# percent of this region's listings that are unique (ie, not duplicates)
listings_ratios['unique_ratio'] = listings_ratios['unique_listings'] / listings_ratios['all_listings']

listings_ratios.head()